In [1]:
# import modules
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import datetime
import pandas as pd
import pandasql as ps

first는 leiden_cluster_biblio_info를 뜻합니다.(맨 처음 완성한 데이터셋)
여기서 필요한 컬럼만 뽑습니다.
uid 유니크를 검사합니다.

In [2]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
first = pd.read_csv('./final/first.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-08 10:17:04.730582
종료: 2024-03-08 10:18:07.896583


In [3]:
selected_first = first[['uid', 'cluster_id', 'pubyear2']]


In [4]:
selected_first = selected_first.rename(columns={'pubyear2': 'pubyear'})


DB의 category_subject 테이블에서 subject_ascatype이 'traditional'인 것만 추출하여 저장한후, 그 파일을 불러옵니다.

In [5]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
sub =  pd.read_csv('./dict/category_subject.txt', sep='\t')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-08 10:18:08.480523
종료: 2024-03-08 10:19:24.293395


In [6]:
selected_first

,uid,cluster_id,pubyear
0,WOS:000241489600007,1,2006
1,WOS:000235430800004,1,2006
2,WOS:000236201900004,1,2006
3,WOS:000237306100004,1,2006
4,WOS:000238242200001,1,2006
...,...,...,...
24339950,WOS:000661166500001,4215,2021
24339951,WOS:000741493600001,4215,2021
24339952,WOS:000592049600001,4215,2021
24339953,WOS:000689390100001,4215,2021


In [7]:
sub

,uid,cate_n,subject_traditional
0,WOS:000201977400052,2,Multidisciplinary Sciences
1,WOS:000200451400481,2,"Medicine, General & Internal"
2,WOS:000205970500001,2,Law
3,WOS:000204963000018,2,Psychiatry
4,WOS:000200960000105,2,Biology
...,...,...,...
138594296,WOS:000883130700065,2,Pharmacology & Pharmacy
138594297,WOS:000787659200001,2,Business
138594298,WOS:000815539500001,2,Economics
138594299,WOS:000823752200014,2,Pharmacology & Pharmacy


In [8]:
now = datetime.datetime.now() 
print('시작:', now)
# 청크와 다른 데이터셋 조인
five_result1 = pd.merge(selected_first, sub, on='uid', how='left')
#result_chunk = pd.merge(df_chunk, third, left_on='uid', right_on='refs_uid', how='inner')
now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-08 10:19:24.319114
종료: 2024-03-08 10:21:28.232880


In [9]:
#sorted_second_result3
# 각 컬럼별 null 값의 개수 계산
null_counts1 = five_result1.isnull().sum()

# 결과 출력
print(null_counts1)

uid                        0
cluster_id                 0
pubyear                    0
cate_n                 21821
subject_traditional    21821
dtype: int64


In [10]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique2 = len(selected_first['uid'].unique()) == len(selected_first)
print("UID 컬럼은 유일한가?", is_unique2)

UID 컬럼은 유일한가? True


결측치 체크를 한 후, 결측치를 드롭합니다.
cate_n과 subject_traditional 의 21821개의 null 값이 존재하여, 해당 행을 드롭했습니다.

In [ ]:
df_cleaned = five_result1.dropna()


중간저장

In [ ]:
now = datetime.datetime.now() 
print('시작:', now)
df_cleaned.to_csv('./mid/uid_sub.txt', sep='\t', index=False)
now = datetime.datetime.now()
print('종료:', now)

In [ ]:
카테고리 유니크 개수를 조사합니다.
총 254개의 카테고리 개수가 나왔습니다.

In [ ]:
unique_subjects = df_cleaned['subject_traditional'].unique()
print(len(unique_subjects))

254개의 카테고리 저장

나중에 wide 버전을 만들때 쓰일 카데고리 리스트를 pickle 파일로 미리 저장해놓습니다.

고유값들을 파일로 저장하고, 이를 나중에 다시 불러와서 DataFrame의 컬럼으로 사용하려는 목적이라면, 텍스트 파일 대신에 Python의 리스트 형태로 직접 저장하고 불러오는 방법이 더 효율적일 수 있습니다. 이 방법은 데이터를 구조화된 형태로 저장하고, 나중에 정확하게 같은 형태로 불러올 수 있게 해줍니다.

Pickle 파일로 저장하고 불러오기
Pickle은 Python 객체를 바이트 스트림으로 직렬화하고, 이를 파일로 저장하여 나중에 다시 불러올 수 있게 하는 라이브러리입니다. 이 방법을 사용하면, 고유값들을 리스트 형태로 직접 저장할 수 있으며, 나중에 이 리스트를 불러와서 즉시 사용할 수 있습니다.



In [ ]:
import pickle

# 고유값 리스트 생성
unique_subjects_list = df_cleaned['subject_traditional'].unique().tolist()

# Pickle 파일로 저장
with open('./mid/unique_subjects.pkl', 'wb') as file:
    pickle.dump(unique_subjects_list, file)


In [ ]:
#import pickle
# Pickle 파일에서 리스트 불러오기
#with open('./mid/unique_subjects.pkl', 'rb') as file:
#    loaded_unique_subjects = pickle.load(file)

# 이제 loaded_unique_subjects 리스트를 사용하여 DataFrame의 컬럼을 생성할 수 있음


# 여기부터 five start

In [2]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
uid_sub = pd.read_csv('./mid/uid_sub.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-06 16:43:41.899134
종료: 2024-03-06 16:44:06.729146


In [6]:
uid_sub

,uid,cluster_id,pubyear,cate_n,subject_traditional
0,WOS:000241489600007,1,2006,2.0,Geochemistry & Geophysics
1,WOS:000235430800004,1,2006,2.0,Geochemistry & Geophysics
2,WOS:000236201900004,1,2006,4.0,Geochemistry & Geophysics
3,WOS:000236201900004,1,2006,4.0,Mineralogy
4,WOS:000237306100004,1,2006,2.0,"Geosciences, Multidisciplinary"
...,...,...,...,...,...
41190333,WOS:000592049600001,4215,2021,6.0,Operations Research & Management Science
41190334,WOS:000689390100001,4215,2021,3.0,"Engineering, Electrical & Electronic"
41190335,WOS:000689390100001,4215,2021,3.0,"Engineering, Mechanical"
41190336,WOS:000664877800004,4215,2021,4.0,"Mathematics, Interdisciplinary Applications"


일대일 대응 체크

In [13]:
# uid 컬럼의 고유한 값들의 수와 데이터 프레임의 행 수 비교
is_unique8 = len(uid_sub['uid'].unique()) == len(uid_sub)
print("UID 컬럼은 유일한가?", is_unique8)

UID 컬럼은 유일한가? False


uid별 subject_category가 몇개인지 나타내주는 sub_n 컬럼을 생성합니다.

In [3]:
uid_sub['sub_n'] = uid_sub.groupby('uid')['subject_traditional'].transform(lambda x: x.nunique())
#df2['cate_n'] = df2.groupby('refs_uid')['subject_traditional'].transform(lambda x: x.nunique())

In [4]:
uid_sub

,uid,cluster_id,pubyear,cate_n,subject_traditional,sub_n
0,WOS:000241489600007,1,2006,2.0,Geochemistry & Geophysics,1
1,WOS:000235430800004,1,2006,2.0,Geochemistry & Geophysics,1
2,WOS:000236201900004,1,2006,4.0,Geochemistry & Geophysics,2
3,WOS:000236201900004,1,2006,4.0,Mineralogy,2
4,WOS:000237306100004,1,2006,2.0,"Geosciences, Multidisciplinary",1
...,...,...,...,...,...,...
41190333,WOS:000592049600001,4215,2021,6.0,Operations Research & Management Science,3
41190334,WOS:000689390100001,4215,2021,3.0,"Engineering, Electrical & Electronic",2
41190335,WOS:000689390100001,4215,2021,3.0,"Engineering, Mechanical",2
41190336,WOS:000664877800004,4215,2021,4.0,"Mathematics, Interdisciplinary Applications",2


In [5]:
now = datetime.datetime.now() 
print('시작:', now)

uid_sub.to_csv('./final/five/temp/mid_result_of_five.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-06 17:28:58.693080
종료: 2024-03-06 17:30:45.853259


check point

In [6]:
five = uid_sub[['cluster_id', 'pubyear', 'uid' ,'subject_traditional', 'sub_n']]



In [7]:
sorted_five = five.sort_values(by=['cluster_id', 'pubyear'], ascending=[True, True])


In [8]:
c_id_counts = sorted_five['cluster_id'].value_counts()
print(c_id_counts)

cluster_id
18      77297
11      74564
7       74324
25      72937
43      72066
        ...  
4065       60
4194       60
4170       53
4166       26
4174       14
Name: count, Length: 4215, dtype: int64


In [9]:
dd_counts = sorted_five['sub_n'].value_counts()
print(dd_counts)

sub_n
2    14484948
1    12951404
3     9113517
4     3437680
5      807725
6      395064
Name: count, dtype: int64


아래의 컬럼을 생성해줍니다.

### w = 1 / (sub_n)
### weight_cate : 한 클러스터 안에서 카테고리별 각각 w를 합산한 값
### norm_weight_cate : weight_cate를 unique한 uid 개수로 나눈 값
(weight_cate, norm_weight_cate 계산시 기준 : all_year는 클러스터별로, by_year는 클러스터별+년도별로 계산)

작업시 느낀점 기록 : weight_cate와 norm_weight_cate를 계산하고, 원래 데이터프레임에 merge해주는데, 이 작업은 불필요한 것 같습니다.

In [10]:
# Assuming sorted_five is your dataframe and it already exists with a column named 'sub_n'
# Example to add a new column 'w' where w = 1 / sub_n

sorted_five['w'] = 1 / sorted_five['sub_n']

In [11]:
sorted_five

,cluster_id,pubyear,uid,subject_traditional,sub_n,w
0,1,2006,WOS:000241489600007,Geochemistry & Geophysics,1,1.000000
1,1,2006,WOS:000235430800004,Geochemistry & Geophysics,1,1.000000
2,1,2006,WOS:000236201900004,Geochemistry & Geophysics,2,0.500000
3,1,2006,WOS:000236201900004,Mineralogy,2,0.500000
4,1,2006,WOS:000237306100004,"Geosciences, Multidisciplinary",1,1.000000
...,...,...,...,...,...,...
41190333,4215,2021,WOS:000592049600001,Operations Research & Management Science,3,0.333333
41190334,4215,2021,WOS:000689390100001,"Engineering, Electrical & Electronic",2,0.500000
41190335,4215,2021,WOS:000689390100001,"Engineering, Mechanical",2,0.500000
41190336,4215,2021,WOS:000664877800004,"Mathematics, Interdisciplinary Applications",2,0.500000


In [12]:
# 클러스터당 uid unique 개수 카운트
# df_example 데이터프레임을 사용하여 cluster_id 별로 uid의 유니크한 개수를 카운트합니다.
uid_counts_per_cluster = sorted_five.groupby('cluster_id')['uid'].nunique().reset_index(name='unique_uid_count')

# 원래 df_example 데이터프레임에 uid_counts_per_cluster 데이터프레임을 merge하여,
# 각 row에 대한 cluster_id별 unique uid 개수를 추가합니다.
sorted_five = pd.merge(sorted_five, uid_counts_per_cluster, on='cluster_id')




In [13]:

# cluster_id와 subject_traditional로 groupby 하고, w 컬럼의 합계를 계산 후,
# 이를 원본 df_example에 merge하여 각 row에 대한 weight_cate 값을 추가합니다.
grouped_five = sorted_five.groupby(['cluster_id', 'subject_traditional'])['w'].sum().reset_index(name='weight_cate')

# 원본 데이터프레임에 weight_cate 값을 merge합니다.
five_result1 = pd.merge(sorted_five, grouped_five, on=['cluster_id', 'subject_traditional'])




In [14]:
five_result1

,cluster_id,pubyear,uid,subject_traditional,sub_n,w,unique_uid_count,weight_cate
0,1,2006,WOS:000241489600007,Geochemistry & Geophysics,1,1.00,52314,13212.75
1,1,2006,WOS:000235430800004,Geochemistry & Geophysics,1,1.00,52314,13212.75
2,1,2006,WOS:000236201900004,Geochemistry & Geophysics,2,0.50,52314,13212.75
3,1,2006,WOS:000236600600022,Geochemistry & Geophysics,1,1.00,52314,13212.75
4,1,2006,WOS:000240601800015,Geochemistry & Geophysics,1,1.00,52314,13212.75
...,...,...,...,...,...,...,...,...
41190333,4215,2020,WOS:000580351000001,"Physics, Applied",4,0.25,71,0.50
41190334,4215,2019,WOS:000511959000012,Metallurgy & Metallurgical Engineering,1,1.00,71,1.00
41190335,4215,2020,WOS:000580351000001,"Chemistry, Analytical",4,0.25,71,0.25
41190336,4215,2020,WOS:000580351000001,Nanoscience & Nanotechnology,4,0.25,71,0.25


In [15]:
five_result1['norm_weight_cate'] = five_result1['weight_cate'] / sorted_five['unique_uid_count']

In [16]:
five_result1

,cluster_id,pubyear,uid,subject_traditional,sub_n,w,unique_uid_count,weight_cate,norm_weight_cate
0,1,2006,WOS:000241489600007,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
1,1,2006,WOS:000235430800004,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
2,1,2006,WOS:000236201900004,Geochemistry & Geophysics,2,0.50,52314,13212.75,0.252566
3,1,2006,WOS:000236600600022,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
4,1,2006,WOS:000240601800015,Geochemistry & Geophysics,1,1.00,52314,13212.75,0.252566
...,...,...,...,...,...,...,...,...,...
41190333,4215,2020,WOS:000580351000001,"Physics, Applied",4,0.25,71,0.50,0.007042
41190334,4215,2019,WOS:000511959000012,Metallurgy & Metallurgical Engineering,1,1.00,71,1.00,0.014085
41190335,4215,2020,WOS:000580351000001,"Chemistry, Analytical",4,0.25,71,0.25,0.003521
41190336,4215,2020,WOS:000580351000001,Nanoscience & Nanotechnology,4,0.25,71,0.25,0.003521


In [17]:
now = datetime.datetime.now() 
print('시작:', now)

five_result1.to_csv('./final/five/temp/mid_result_of_five2.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-06 17:40:47.726614
종료: 2024-03-06 17:44:11.254514


1. final_result1에서 cluster_id와 subject_tra를 groupby하기  
2.필요 컬럼만 추출하여 저장 = 5-1번

In [23]:

# 'five_result1' 데이터프레임에서 'cluster_id'와 'subject_traditional'로 groupby한 후, 각 그룹의 첫 번째 행을 선택
grouped_df = five_result1.groupby(['cluster_id', 'subject_traditional']).first().reset_index()

In [24]:
grouped_df

,cluster_id,subject_traditional,pubyear,uid,sub_n,w,unique_uid_count,weight_cate,norm_weight_cate
0,1,Acoustics,2021,WOS:000670625600004,3,0.333333,52314,0.333333,0.000006
1,1,Anthropology,2006,WOS:000241072600005,2,0.500000,52314,3.833333,0.000073
2,1,Archaeology,2006,WOS:000241072600005,2,0.500000,52314,7.833333,0.000150
3,1,Art,2015,WOS:000355644100018,1,1.000000,52314,1.000000,0.000019
4,1,Asian Studies,2018,WOS:000632524500009,2,0.500000,52314,0.500000,0.000010
...,...,...,...,...,...,...,...,...,...
391113,4215,Multidisciplinary Sciences,2021,WOS:000664877800004,2,0.500000,71,0.500000,0.007042
391114,4215,Nanoscience & Nanotechnology,2020,WOS:000580351000001,4,0.250000,71,0.250000,0.003521
391115,4215,Operations Research & Management Science,2008,WOS:000258591600005,3,0.333333,71,1.000000,0.014085
391116,4215,"Physics, Applied",2019,WOS:000472641200077,4,0.250000,71,0.500000,0.007042


In [25]:
now = datetime.datetime.now() 
print('시작:', now)

five_result1.to_csv('./final/five/temp/mid_result_of_five3.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-06 17:51:01.926553
종료: 2024-03-06 17:54:23.322330


In [26]:
# Selecting specific columns from the grouped DataFrame
selected_columns_df = grouped_df[['cluster_id', 'subject_traditional', 'weight_cate', 'norm_weight_cate']]



In [27]:
now = datetime.datetime.now() 
print('시작:', now)

selected_columns_df.to_csv('./final/five/temp/mid_result_of_five4.txt', sep='\t', index=False)

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-06 17:54:23.341515
종료: 2024-03-06 17:54:24.637806


In [2]:
# reading 
now = datetime.datetime.now() 
print('시작:', now)
first = pd.read_csv('./final/five/temp/mid_result_of_five4.txt', sep='\t')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-07 09:23:38.167399
종료: 2024-03-07 09:23:38.357442


In [3]:
first

,cluster_id,subject_traditional,weight_cate,norm_weight_cate
0,1,Acoustics,0.333333,0.000006
1,1,Anthropology,3.833333,0.000073
2,1,Archaeology,7.833333,0.000150
3,1,Art,1.000000,0.000019
4,1,Asian Studies,0.500000,0.000010
...,...,...,...,...
391113,4215,Multidisciplinary Sciences,0.500000,0.007042
391114,4215,Nanoscience & Nanotechnology,0.250000,0.003521
391115,4215,Operations Research & Management Science,1.000000,0.014085
391116,4215,"Physics, Applied",0.500000,0.007042


In [4]:
unique_counts = first.groupby('cluster_id')['subject_traditional'].nunique()

# 결과 출력
print(unique_counts)

cluster_id
1        93
2        57
3       205
4       106
5       166
       ... 
4211     40
4212     48
4213     27
4214     57
4215     24
Name: subject_traditional, Length: 4215, dtype: int64


In [6]:
import pickle
# Pickle 파일에서 리스트 불러오기
with open('./mid/unique_subjects.pkl', 'rb') as file:
    loaded_unique_subjects = pickle.load(file)

# 이제 loaded_unique_subjects 리스트를 사용하여 DataFrame의 컬럼을 생성할 수 있음


In [8]:
print(len(loaded_unique_subjects))

254


In [11]:
loaded_unique_subjects

['Geochemistry & Geophysics',
 'Mineralogy',
 'Geosciences, Multidisciplinary',
 'Multidisciplinary Sciences',
 'Geology',
 'Mining & Mineral Processing',
 'Astronomy & Astrophysics',
 'Materials Science, Multidisciplinary',
 'Physics, Atomic, Molecular & Chemical',
 'Spectroscopy',
 'Chemistry, Analytical',
 'Chemistry, Multidisciplinary',
 'Instruments & Instrumentation',
 'Meteorology & Atmospheric Sciences',
 'Water Resources',
 'Computer Science, Interdisciplinary Applications',
 'Paleontology',
 'Microscopy',
 'Biochemical Research Methods',
 'Environmental Sciences',
 'Geography, Physical',
 'Chemistry, Applied',
 'Anthropology',
 'Archaeology',
 'Nuclear Science & Technology',
 'Physics, Nuclear',
 'Crystallography',
 'Chemistry, Inorganic & Nuclear',
 'Chemistry, Physical',
 'Materials Science, Coatings & Films',
 'Physics, Applied',
 'Physics, Condensed Matter',
 'Radiology, Nuclear Medicine & Medical Imaging',
 'Metallurgy & Metallurgical Engineering',
 'Engineering, Chemica

1. 254 * 4215 뼈대 만들기  
2. left join하기 - 결측값은 0으로 채우기  

In [15]:
import pandas as pd
import numpy as np

# cluster_id 생성: 1부터 4215까지
cluster_ids = np.arange(1, 4216)

In [16]:
print(cluster_ids)

[   1    2    3 ... 4213 4214 4215]


In [18]:
subject_traditional = loaded_unique_subjects

In [19]:
print(len(subject_traditional))

254


In [20]:
# DataFrame 생성
df = pd.DataFrame({
    'cluster_id': np.repeat(cluster_ids, len(subject_traditional)),
    'subject_traditional': np.tile(subject_traditional, len(cluster_ids))
})

In [21]:
df

,cluster_id,subject_traditional
0,1,Geochemistry & Geophysics
1,1,Mineralogy
2,1,"Geosciences, Multidisciplinary"
3,1,Multidisciplinary Sciences
4,1,Geology
...,...,...
1070605,4215,"Psychology, Psychoanalysis"
1070606,4215,Ornithology
1070607,4215,"Literature, German, Dutch, Scandinavian"
1070608,4215,"Literature, Slavic"


In [22]:
unique_counts = df.groupby('cluster_id')['subject_traditional'].nunique()

# 결과 출력
print(unique_counts)

cluster_id
1       254
2       254
3       254
4       254
5       254
       ... 
4211    254
4212    254
4213    254
4214    254
4215    254
Name: subject_traditional, Length: 4215, dtype: int64


In [23]:

now = datetime.datetime.now() 
print('시작:', now)

result= pd.merge(df, first, on=['cluster_id','subject_traditional'], how='left')

now = datetime.datetime.now()
print('종료:', now)

시작: 2024-03-07 10:02:02.201683
종료: 2024-03-07 10:02:02.474144


In [24]:
result

,cluster_id,subject_traditional,weight_cate,norm_weight_cate
0,1,Geochemistry & Geophysics,13212.750000,0.252566
1,1,Mineralogy,7064.416667,0.135039
2,1,"Geosciences, Multidisciplinary",19794.333333,0.378375
3,1,Multidisciplinary Sciences,1061.000000,0.020281
4,1,Geology,8930.333333,0.170706
...,...,...,...,...
1070605,4215,"Psychology, Psychoanalysis",NaN,NaN
1070606,4215,Ornithology,NaN,NaN
1070607,4215,"Literature, German, Dutch, Scandinavian",NaN,NaN
1070608,4215,"Literature, Slavic",NaN,NaN


In [25]:
result.fillna(0, inplace=True)

In [26]:
result

,cluster_id,subject_traditional,weight_cate,norm_weight_cate
0,1,Geochemistry & Geophysics,13212.750000,0.252566
1,1,Mineralogy,7064.416667,0.135039
2,1,"Geosciences, Multidisciplinary",19794.333333,0.378375
3,1,Multidisciplinary Sciences,1061.000000,0.020281
4,1,Geology,8930.333333,0.170706
...,...,...,...,...
1070605,4215,"Psychology, Psychoanalysis",0.000000,0.000000
1070606,4215,Ornithology,0.000000,0.000000
1070607,4215,"Literature, German, Dutch, Scandinavian",0.000000,0.000000
1070608,4215,"Literature, Slavic",0.000000,0.000000


In [28]:
# 'cluster_id'를 우선적으로, 그리고 'subject_traditional'을 알파벳 순으로 정렬
result = result.sort_values(by=['cluster_id', 'subject_traditional'])


In [30]:
result.to_csv('./final/five/temp/mid_result_of_five5.txt', sep='\t', index=False)

In [32]:
result

,cluster_id,subject_traditional,weight_cate,norm_weight_cate
92,1,Acoustics,0.333333,0.000006
240,1,Agricultural Economics & Policy,0.000000,0.000000
232,1,Agricultural Engineering,0.000000,0.000000
159,1,"Agriculture, Dairy & Animal Science",0.000000,0.000000
161,1,"Agriculture, Multidisciplinary",0.000000,0.000000
...,...,...,...,...
1070489,4215,Veterinary Sciences,0.000000,0.000000
1070466,4215,Virology,0.000000,0.000000
1070370,4215,Water Resources,0.000000,0.000000
1070561,4215,Women's Studies,0.000000,0.000000


In [38]:

# pivot_table을 사용하여 wide 형식의 데이터프레임 생성
pivot_df = result.pivot_table(index='cluster_id', columns='subject_traditional', values='norm_weight_cate', fill_value=0)

# 컬럼 순서 조정을 위한 준비: 'pubyear' 컬럼 추가
pivot_df['pubyear'] = 'all'

# 원하는 컬럼 순서 생성: 'cluster_id' 다음에 'pubyear'가 오도록
new_columns_order = ['pubyear'] + [col for col in pivot_df.columns if col not in [ 'pubyear']]

# DataFrame의 컬럼을 새 순서로 재배열
pivot_df_1 = pivot_df[new_columns_order]

# 인덱스에 이름을 지정
pivot_df_1.index.name = 'cluster_id'


pivot_df_1.to_csv('./final/five/temp/mid_result_of_five6.txt', sep='\t', index=True)